**Benefits od paralllel:**

-- having new datasets we can easly imporve the model


In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [172]:
pd.__version__
# csv.__version__

'0.24.2'

Load datasets

In [3]:
cleveland = pd.read_csv("data/processed.cleveland.data", header=None)
hungarian = pd.read_csv("data/processed.hungarian.data", header=None)
switzerland = pd.read_csv("data/processed.switzerland.data", header=None)
va = pd.read_csv("data/processed.va.data", header=None)

In [4]:
columns = ["Age", "Sex", "ChPain", "RestBlPrs", "SerChol", "FBlSug", "RestECG", "MaxHR", "Eia", "STdep", "PeakExST", "MajVess", "Thal", "HeartDis"]

In [5]:
cleveland.columns = columns
hungarian.columns = columns
switzerland.columns = columns
va.columns = columns

In [6]:
def check_missing_values(df, columns):
    indx = []
    for col in columns:
        tmp = df[df[col] == "?"][col]
        indx.extend(tmp.index)
#         print(col, tmp.count())
    return indx

In [7]:
idx = check_missing_values(cleveland, columns)
# cleveland.iloc[idx]

/home/jb/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [8]:
# remove rows with missing data from cleveland dataset
data = cleveland.loc[~cleveland.index.isin(idx)]

#TODO: change most of the colmns to integer type
data = data.apply(pd.to_numeric) 
# data.dtypes

In [9]:
y = data["HeartDis"]

label_mapping = {0:-1, 1:1, 2:1, 3:1, 4:1}
y = [label_mapping[label] for label in y]

X = data.loc[:, data.columns != 'HeartDis']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)

In [11]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Parallel Sharing-based SVM

**Divide data into blocks - splitting of features**

In [51]:
def get_feature_chunks(A, x, n):
    """
    Split data by columns.
    
    n - the number of columns in a chunk.
        When data cannot be divided into chunks of the same length,
        a last chunk will be uncomplete.
    """
    sections = range(0, A.shape[1], n)[1:]
    A_chunks = np.hsplit(A, sections)
    x_chunks = np.hsplit(x, sections)
    return A_chunks, x_chunks

In [52]:
def generate_A_mat(X, y):
    """
    Generates the A matrix for parallel SVM.
    
    X is a matrix of shape (N x m);
    y is a vector of length N;
    A is a matrix of shape (N x (m+1));
    A = [a1 a2 ... ai ... aN].T,  
    where ai = [-yi*xi -yi]
    """
    # Add one at the end of all x-data vectors
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1)
    
    # -1 * y_vector
    minus_y = np.multiply(-1, y).reshape(-1, 1)
    
    # Componentiwise multiplication
    A = np.multiply(minus_y, X)
    return A

### ADMM

In [86]:
class Node():
    
    def __init__(self, A_chunk, x, mean_Ax, rho, lamb):
        self.A = A_chunk
        self.x = x.reshape(-1,1)
        self.rho = rho
        self.lamb = lamb
        self.mean_Ax = mean_Ax.reshape(-1,1)
    
    def compute_x_i(self, u, z):
        const = 2*self.lamb/self.rho
        AA = self.A.T.dot(self.A)
        part1 = np.linalg.inv(const + AA)
        
        Ax = self.A.dot(self.x)
        z = z.reshape(-1,1)
        u = u.reshape(-1,1)
        part2 = self.A.T.dot(Ax - self.mean_Ax + z - u)
        
        self.x = part1.dot(part2)

In [157]:
class FusionCenter():
    
    def __init__(self, N, rho, lamb, A_chunks, x_chunks, nodes):
        self.N = N
        self.rho = rho
        self.lamb = lamb
        self.A_chunks = A_chunks
        self.x_chunks = x_chunks
        self.nodes = nodes #to observe all Node instances
        self.N = self.A_chunks[0].shape[0]
    
    def collect_x(self):
        """ Collect x_chunk from every Node"""
        self.old_x_chunks = self.x_chunks.copy()
        self.x_chunks = [node.x for node in self.nodes]


    def _average_Ax(self):
        """ Calculate all Ai*xi and 
            return the componentwise average ?? """
        
        Ax_all = np.zeros((self.N, len(x_chunks)))

        for idx, A in enumerate(self.A_chunks):
            x = np.array(self.x_chunks[idx]).reshape(-1,1)
            Ax_all[:, idx] = A.dot(x).T

        return np.average(Ax_all, axis=1).reshape(-1,1)

            
    def _average_x(self):
        x = np.array(self.x_list) 
        self.x_bar = np.average(x, axis=0).reshape(-1,1)
    
    def compute_z(self, u):
        self.Ax_bar = self._average_Ax()
        v = self.Ax_bar + u
        self.z = np.zeros((self.N,1))
        for idx, vi in enumerate(v):
            if vi > 1/self.rho - (1/self.N):
                self.z[idx] = vi - 1/self.rho
                
            elif vi < -1/self.N:
                self.z[idx] = vi
            
            else:
                self.z[idx] = -1/self.N
    
    def compute_u(self, u):
        self.u = u + self.Ax_bar - self.z
        
    def check_convergence(self, error_tol):
        old = np.concatenate(self.old_x_chunks)
        new = np.concatenate(self.x_chunks)
        print(abs(old - new))
        if (abs(old - new) > error_tol).any():
            return False
        else:
            return True

### def __main__:

In [158]:
# Fixed parameters:
RHO = 0.1
LAMBD = 0.1
E_TOL = 0.000001

In [159]:
# First iteration:
N, m = X_train.shape
z_init = np.zeros((N,1))
u_init = np.zeros((N,1))
x_init = np.zeros(m+1)

A = generate_A_mat(X_train, y_train)
A_chunks, x_chunks = get_feature_chunks(A, x_init, 5)
mean_Ax_init = np.zeros((N,1))
N = len(A_chunks)

In [160]:
node1 = Node(A_chunk=A_chunks[0], x=x_chunks[0], mean_Ax=mean_Ax_init,
             rho = RHO, lamb=LAMBD)
node2 = Node(A_chunk=A_chunks[1], x=x_chunks[1], mean_Ax=mean_Ax_init,
             rho = RHO, lamb=LAMBD)
node3 = Node(A_chunk=A_chunks[2], x=x_chunks[2], mean_Ax=mean_Ax_init,
             rho = RHO, lamb=LAMBD)

FC = FusionCenter(N=N, rho=RHO, lamb=LAMBD,
                  A_chunks = A_chunks,
                  x_chunks = x_chunks,
                  nodes = [node1, node2, node3])

############################################################

node1.compute_x_i(u = u_init, z = z_init)
node2.compute_x_i(u = u_init, z = z_init)
node3.compute_x_i(u = u_init, z = z_init)

FC.collect_x()

In [162]:
# More iterations

stop=False

while not stop:
    FC.compute_z(u = u_init)
    FC.compute_u(u = u_init)
    
    node1.compute_x_i(u = FC.u, z = FC.z)
    node2.compute_x_i(u = FC.u, z = FC.z)
    node3.compute_x_i(u = FC.u, z = FC.z)
    
    FC.collect_x()
    stop = FC.check_convergence(error_tol=E_TOL)

[[0.00127028]
 [0.00241751]
 [0.00295959]
 [0.000922  ]
 [0.00074609]
 [0.00024092]
 [0.00130133]
 [0.00199416]
 [0.00199059]
 [0.00241928]
 [0.00161215]
 [0.0029617 ]
 [0.00295253]
 [0.00066323]]
[[0.00068796]
 [0.0014935 ]
 [0.00172113]
 [0.0006736 ]
 [0.00067384]
 [0.00035384]
 [0.00092794]
 [0.00121363]
 [0.00120172]
 [0.00159266]
 [0.0008824 ]
 [0.00214578]
 [0.00163749]
 [0.0003059 ]]
[[0.00138204]
 [0.00239496]
 [0.00247331]
 [0.00106461]
 [0.00102369]
 [0.0001998 ]
 [0.00136258]
 [0.00190004]
 [0.00160087]
 [0.00254772]
 [0.0014682 ]
 [0.00304596]
 [0.00231454]
 [0.0001667 ]]
[[2.53112372e-03]
 [3.71024347e-03]
 [3.93475932e-03]
 [1.57204348e-03]
 [1.55993525e-03]
 [6.92301225e-06]
 [2.26540511e-03]
 [3.12004139e-03]
 [2.41118799e-03]
 [4.14211841e-03]
 [2.57363591e-03]
 [4.64738378e-03]
 [3.56689608e-03]
 [6.17213628e-05]]
[[4.49997246e-03]
 [5.95359015e-03]
 [6.28834440e-03]
 [2.26375764e-03]
 [2.41536217e-03]
 [3.27609940e-04]
 [3.73174672e-03]
 [5.27225997e-03]
 [3.74420496

[[8.82398581e+31]
 [7.64906846e+31]
 [8.96809618e+31]
 [1.47922638e+31]
 [2.08954979e+30]
 [2.34879852e+31]
 [3.10932823e+31]
 [9.97201293e+31]
 [3.56082368e+31]
 [9.36805313e+31]
 [8.03473402e+31]
 [8.16287308e+31]
 [8.24348366e+31]
 [9.28132187e+30]]
[[1.49645175e+32]
 [1.29719858e+32]
 [1.52089130e+32]
 [2.50860661e+31]
 [3.54364854e+30]
 [3.98330611e+31]
 [5.27308156e+31]
 [1.69114463e+32]
 [6.03876859e+31]
 [1.58871964e+32]
 [1.36260326e+32]
 [1.38433424e+32]
 [1.39800492e+32]
 [1.57401096e+31]]
[[2.53781896e+32]
 [2.19990732e+32]
 [2.57926576e+32]
 [4.25432321e+31]
 [6.00964143e+30]
 [6.75525271e+31]
 [8.94257120e+31]
 [2.86799684e+32]
 [1.02410929e+32]
 [2.69429523e+32]
 [2.31082651e+32]
 [2.34767989e+32]
 [2.37086387e+32]
 [2.66935092e+31]]
[[4.30386416e+32]
 [3.73080287e+32]
 [4.37415342e+32]
 [7.21486815e+31]
 [1.01916964e+31]
 [1.14561718e+32]
 [1.51656254e+32]
 [4.86380984e+32]
 [1.73677766e+32]
 [4.56923085e+32]
 [3.91890972e+32]
 [3.98140904e+32]
 [4.02072653e+32]
 [4.526

[[1.95331042e+64]
 [1.69322633e+64]
 [1.98521123e+64]
 [3.27447071e+63]
 [4.62550538e+62]
 [5.19938803e+63]
 [6.88292499e+63]
 [2.20744199e+64]
 [7.88237215e+63]
 [2.07374719e+64]
 [1.77859870e+64]
 [1.80696404e+64]
 [1.82480830e+64]
 [2.05454804e+63]]
[[3.31260143e+64]
 [2.87152718e+64]
 [3.36670173e+64]
 [5.55314519e+63]
 [7.84435263e+62]
 [8.81759501e+63]
 [1.16726901e+64]
 [3.74358085e+64]
 [1.33676435e+64]
 [3.51684906e+64]
 [3.01630941e+64]
 [3.06441393e+64]
 [3.09467585e+64]
 [3.48428939e+63]]
[[5.61781073e+64]
 [4.86979692e+64]
 [5.70955893e+64]
 [9.41752858e+63]
 [1.33031665e+63]
 [1.49536794e+64]
 [1.97956093e+64]
 [6.34870482e+64]
 [2.26700653e+64]
 [5.96419244e+64]
 [5.11533178e+64]
 [5.19691180e+64]
 [5.24823271e+64]
 [5.90897478e+63]]
[[9.52719428e+64]
 [8.25864443e+64]
 [9.68278922e+64]
 [1.59711013e+64]
 [2.25607194e+63]
 [2.53598094e+64]
 [3.35711942e+64]
 [1.07667109e+65]
 [3.84459582e+64]
 [1.01146199e+65]
 [8.67504478e+64]
 [8.81339560e+64]
 [8.90043027e+64]
 [1.002

[[1.07174707e+95]
 [9.29043507e+94]
 [1.08925048e+95]
 [1.79664449e+94]
 [2.53793344e+93]
 [2.85281276e+94]
 [3.77653988e+94]
 [1.21118459e+95]
 [4.32491896e+94]
 [1.13782861e+95]
 [9.75885824e+94]
 [9.91449387e+94]
 [1.00124022e+95]
 [1.12729437e+94]]
[[1.81756614e+95]
 [1.57555646e+95]
 [1.84725002e+95]
 [3.04691309e+94]
 [4.30405831e+93]
 [4.83805929e+94]
 [6.40459974e+94]
 [2.05403697e+95]
 [7.33459084e+94]
 [1.92963322e+95]
 [1.65499592e+95]
 [1.68139003e+95]
 [1.69799421e+95]
 [1.91176830e+94]]
[[3.08239394e+95]
 [2.67197191e+95]
 [3.13273456e+95]
 [5.16723229e+94]
 [7.29921347e+93]
 [8.20482089e+94]
 [1.08615026e+95]
 [3.48342267e+95]
 [1.24386661e+95]
 [3.27244749e+95]
 [2.80669257e+95]
 [2.85145410e+95]
 [2.87961298e+95]
 [3.24215053e+94]]
[[5.22740393e+95]
 [4.53137294e+95]
 [5.31277614e+95]
 [8.76306239e+94]
 [1.23786699e+94]
 [1.39144814e+95]
 [1.84199238e+95]
 [5.90750492e+95]
 [2.10946211e+95]
 [5.54971402e+95]
 [4.75984449e+95]
 [4.83575515e+95]
 [4.88350954e+95]
 [5.498

[[3.19419407e+131]
 [2.76888581e+131]
 [3.24636058e+131]
 [5.35465066e+130]
 [7.56395998e+129]
 [8.50241428e+130]
 [1.12554553e+131]
 [3.60976834e+131]
 [1.28898234e+131]
 [3.39114097e+131]
 [2.90849287e+131]
 [2.95487792e+131]
 [2.98405814e+131]
 [3.35974512e+130]]
[[5.41700476e+131]
 [4.69572835e+131]
 [5.50547347e+131]
 [9.08090351e+130]
 [1.28276511e+130]
 [1.44191673e+131]
 [1.90880246e+131]
 [6.12177339e+131]
 [2.18597346e+131]
 [5.75100522e+131]
 [4.93248668e+131]
 [5.01115067e+131]
 [5.06063713e+131]
 [5.69776129e+130]]
[[9.18664926e+131]
 [7.96344312e+131]
 [9.33668253e+131]
 [1.54002219e+131]
 [2.17542973e+130]
 [2.44533351e+131]
 [3.23712079e+131]
 [1.03818600e+132]
 [3.70717258e+131]
 [9.75307761e+131]
 [8.36495944e+131]
 [8.49836498e+131]
 [8.58228863e+131]
 [9.66278171e+130]]
[[1.55795552e+132]
 [1.35051310e+132]
 [1.58339952e+132]
 [2.61170967e+131]
 [3.68929155e+130]
 [4.14701892e+131]
 [5.48980380e+131]
 [1.76065022e+132]
 [6.28696036e+131]
 [1.65401559e+132]
 [1.41860

...

[[1.92180602e+181]
 [1.66591675e+181]
 [1.95319231e+181]
 [3.22165768e+180]
 [4.55090187e+179]
 [5.11552853e+180]
 [6.77191219e+180]
 [2.17183876e+181]
 [7.75523954e+180]
 [2.04030030e+181]
 [1.74991217e+181]
 [1.77782002e+181]
 [1.79537647e+181]
 [2.02141080e+180]]
[[3.25917340e+181]
 [2.82521311e+181]
 [3.31240112e+181]
 [5.46358004e+180]
 [7.71783323e+179]
 [8.67537845e+180]
 [1.14844245e+181]
 [3.68320167e+181]
 [1.31520404e+181]
 [3.46012677e+181]
 [2.96766019e+181]
 [3.01498884e+181]
 [3.04476268e+181]
 [3.42809224e+180]]
[[5.52720261e+181]
 [4.79125330e+181]
 [5.61747103e+181]
 [9.26563586e+180]
 [1.30886034e+180]
 [1.47124957e+181]
 [1.94763313e+181]
 [6.24630831e+181]
 [2.23044261e+181]
 [5.86799762e+181]
 [5.03282800e+181]
 [5.11309224e+181]
 [5.16358541e+181]
 [5.81367055e+180]]
[[9.37353279e+181]
 [8.12544303e+181]
 [9.52661818e+181]
 [1.57135078e+181]
 [2.21968438e+180]
 [2.49507880e+181]
 [3.30297337e+181]
 [1.05930576e+182]
 [3.78258740e+181]
 [9.95148396e+181]
 [8.53512

[[2.40763713e+216]
 [2.08705926e+216]
 [2.44695785e+216]
 [4.03609032e+215]
 [5.70136645e+214]
 [6.40873030e+215]
 [8.48384650e+215]
 [2.72087797e+216]
 [9.71575827e+215]
 [2.55608668e+216]
 [2.19228866e+216]
 [2.22725158e+216]
 [2.24924629e+216]
 [2.53242193e+215]]
[[4.08308998e+216]
 [3.53942487e+216]
 [4.14977364e+216]
 [6.84476898e+215]
 [9.66889567e+214]
 [1.08685076e+216]
 [1.43876784e+216]
 [4.61431228e+216]
 [1.64768664e+216]
 [4.33484422e+216]
 [3.71788245e+216]
 [3.77717576e+216]
 [3.81447639e+216]
 [4.29471139e+215]]
[[6.92447528e+216]
 [6.00247854e+216]
 [7.03756348e+216]
 [1.16079816e+216]
 [1.63973925e+215]
 [1.84318035e+216]
 [2.43999333e+216]
 [7.82537036e+216]
 [2.79429683e+216]
 [7.35142302e+216]
 [6.30512314e+216]
 [6.40567812e+216]
 [6.46893593e+216]
 [7.28336210e+215]]
[[1.17431548e+217]
 [1.01795489e+217]
 [1.19349401e+217]
 [1.96858708e+216]
 [2.78081893e+215]
 [3.12583285e+216]
 [4.13796256e+216]
 [1.32709746e+217]
 [4.73882265e+216]
 [1.24672116e+217]
 [1.06928

[[2.15022015e+250]
 [1.86391745e+250]
 [2.18533682e+250]
 [3.60456424e+249]
 [5.09179429e+248]
 [5.72352902e+249]
 [7.57678032e+249]
 [2.42997026e+250]
 [8.67697996e+249]
 [2.28279793e+250]
 [1.95789605e+250]
 [1.98912085e+250]
 [2.00876395e+250]
 [2.26166334e+249]]
[[3.64653885e+250]
 [3.16100070e+250]
 [3.70609289e+250]
 [6.11294781e+249]
 [8.63512776e+248]
 [9.70648097e+249]
 [1.28493930e+250]
 [4.12096453e+250]
 [1.47152116e+250]
 [3.87137632e+250]
 [3.32037816e+250]
 [3.37333201e+250]
 [3.40664458e+250]
 [3.83553436e+249]]
[[6.18413217e+250]
 [5.36071242e+250]
 [6.28512934e+250]
 [1.03668928e+250]
 [1.46442349e+249]
 [1.64611331e+250]
 [2.17911692e+250]
 [6.98870638e+250]
 [2.49553940e+250]
 [6.56543199e+250]
 [5.63099921e+250]
 [5.72080317e+250]
 [5.77729765e+250]
 [6.50464794e+249]]
[[1.04876137e+251]
 [9.09118358e+250]
 [1.06588939e+251]
 [1.75811194e+250]
 [2.48350251e+249]
 [2.79162863e+250]
 [3.69554462e+250]
 [1.18520838e+251]
 [4.23216264e+250]
 [1.11342566e+251]
 [9.54956

[[5.75436916e+281]
 [4.98817253e+281]
 [5.84834758e+281]
 [9.64645101e+280]
 [1.36265415e+280]
 [1.53171753e+281]
 [2.02768033e+281]
 [6.50302991e+281]
 [2.32211320e+281]
 [6.10917075e+281]
 [5.23967588e+281]
 [5.32323897e+281]
 [5.37580739e+281]
 [6.05261086e+280]]
[[9.75878248e+281]
 [8.45939657e+281]
 [9.91815963e+281]
 [1.63593288e+281]
 [2.31091281e+280]
 [2.59762587e+281]
 [3.43872469e+281]
 [1.10284295e+282]
 [3.93805073e+281]
 [1.03604873e+282]
 [8.88591882e+281]
 [9.02763269e+281]
 [9.11678301e+281]
 [1.02645679e+281]]
[[1.65498307e+282]
 [1.43462139e+282]
 [1.68201170e+282]
 [2.77436374e+281]
 [3.91905609e+280]
 [4.40529016e+281]
 [5.83170202e+281]
 [1.87030135e+282]
 [6.67850452e+281]
 [1.75702564e+282]
 [1.50695492e+282]
 [1.53098804e+282]
 [1.54610696e+282]
 [1.74075876e+281]]
[[2.80667079e+282]
 [2.43296141e+282]
 [2.85250839e+282]
 [4.70501831e+281]
 [6.64629168e+280]
 [7.47089164e+281]
 [9.88993059e+281]
 [3.17182710e+282]
 [1.13260153e+282]
 [2.97972386e+282]
 [2.55563

/home/jb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in greater


**PREDICTIONS and VALIDATION**

In [170]:
# collect weights
x = np.concatenate(FC.x_chunks)
w = x[:-1]
b = x[-1][0]

In [24]:
def predict(X, w, b):
    return np.sign(w.T.flatten().dot(X.T) - b)

In [25]:
y_pred = predict(X_train, w, b)

In [26]:
# accuracy
print("train_acc", accuracy_score(y_train, y_pred))
print("test_acc", accuracy_score(y_test, predict(X_test, w, b)))

train_acc 0.8143459915611815
test_acc 0.8


**WHat about sklearn ?**

In [27]:
from sklearn.svm import SVC

In [28]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
sklearn_pred = clf.predict(X_train)

In [29]:
print("train_acc", accuracy_score(y_train, sklearn_pred))
print("test_acc", accuracy_score(y_test, clf.predict(X_test)))

train_acc 0.9071729957805907
test_acc 0.85


### Visualisations

In [ ]:
#TODO: 1)Accuracy vs. number of training samples

In [ ]:
# 2)